In [3]:
import re
import pickle
from collections import Counter
import pandas as pd

from gensim.models.word2vec import Word2Vec

C:\Users\losya\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [4]:
class SpellChecker:
    def __init__(self, language):
        self.model = Word2Vec.load('newmodel')
        self.language = language
        self.make_vocabulary(self.model)
        
    def make_vocabulary(self, model):
        words = model.wv.index2word
        
        w_rank = {}
        for i,word in enumerate(words):
            w_rank[word] = i

        self.WORDS = w_rank
        
    
    def P(self, word): 
        return - self.WORDS.get(word, 0)

    def correction(self, word): 
        "Most probable spelling correction for word."
        return max(self.candidates(word), key=self.P)

    def candidates(self, word): 
        "Generate possible spelling corrections for word."
        return (self.known([word]) or self.known(self.edits1(word)) or self.known(self.edits2(word)) or [word])

    def known(self, words): 
        "The subset of `words` that appear in the dictionary of WORDS."
        return set(w for w in words if w in self.WORDS)

    def edits1(self, word):
        "All edits that are one edit away from `word`."
        #letters    = 'abcdefghijklmnopqrstuvwxyz'
        letters = 'абвгдежзийклмнопрстуфхцчшщъыьэюя'
        splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
        deletes    = [L + R[1:]               for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
        replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
        inserts    = [L + c + R               for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)

    def edits2(self, word): 
        "All edits that are two edits away from `word`."
        return (e2 for e1 in self.edits1(word) for e2 in self.edits1(e1))
    

In [5]:
sc = SpellChecker('russian')

FileNotFoundError: [Errno 2] No such file or directory: 'newmodel'

In [35]:
sc.model.similarity('париж', 'франция')

C:\Users\busov\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.
C:\Users\busov\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.36117578

In [37]:
from textblob.classifiers import NaiveBayesClassifier as NBC
from textblob import TextBlob

In [67]:
df = pd.read_csv('news_lenta.csv')

In [157]:
df.loc[df['tags'].isna(),'tags'] = 'Неизвестно'

In [255]:
data = df.loc[:50000, ['text', 'tags']]

In [123]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from nltk.corpus import stopwords

import pandas, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [256]:
encoder = preprocessing.LabelEncoder()
data['tags_encoded'] = encoder.fit_transform(data['tags'])

In [257]:
train_x, test_x, train_y, test_y =  model_selection.train_test_split(data['text'], data['tags_encoded'])

In [258]:
tfidf = TfidfVectorizer(stop_words=stopwords.words('russian'), token_pattern=r'\w{1,}', max_features=5000)
tfidf.fit(train_x)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=5000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', '...гда', 'лучше', 'чуть', 'том', 'нельзя', 'такой', 'им', 'более', 'всегда', 'конечно', 'всю', 'между'],
        strip_accents=None, sublinear_tf=False, token_pattern='\\w{1,}',
        tokenizer=None, use_idf=True, vocabulary=None)

In [259]:
xtrain_tfidf = tfidf.transform(train_x)
xtest_tfidf = tfidf.transform(test_x)

In [ ]:
model = linear_model.LogisticRegression(C=30, class_weight='balanced')
model.fit(xtrain_tfidf, train_y)

In [261]:
y_pred = model.predict(xtest_tfidf)

In [262]:
print(metrics.classification_report(y_pred, test_y))

             precision    recall  f1-score   support

          0       0.47      0.39      0.43       175
          1       0.70      0.94      0.80        17
          2       0.67      0.67      0.67         9
          3       0.69      0.52      0.59        21
          4       0.43      0.40      0.41       290
          5       0.92      0.96      0.94       111
          6       0.06      0.17      0.08         6
          7       0.44      0.38      0.41        58
          8       0.20      0.50      0.29         2
          9       0.58      0.67      0.62        76
         11       0.46      0.52      0.49       113
         13       0.78      0.80      0.79        75
         14       0.68      0.68      0.68       565
         15       0.84      0.79      0.82        67
         16       0.43      0.40      0.41       158
         17       0.36      0.28      0.31        89
         18       0.11      0.16      0.13        37
         19       0.06      0.10      0.07   

C:\Users\busov\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
